In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
cal_len = 5

In [3]:
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for idx, char in enumerate(chars)}

In [4]:
# 데이터 생성 함수
def generate_data(num_samples):
    X = []
    y = []
    for _ in range(num_samples):
        sequence = np.random.choice(list(chars), 5)
        X.append([char_to_index[char] for char in sequence])
        y.append([char_to_index[char] for char in sequence[::-1]])
    return np.array(X), np.array(y)

In [5]:
# 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)

In [6]:
# 데이터 형태 변환 (LSTM을 사용하기 위해 3D 텐서로 변환)
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

In [7]:
# 인코더 정의
encoder_inputs = Input(shape = (5, 1))
encoder = LSTM(128, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [8]:
# 디코더 정의
decoder_inputs = Input(shape = (5, 1))
decoder_lstm = LSTM(128, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_classes, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [9]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [10]:
# 입력 데이터와 타겟 데이터를 동일하게 맞춤
decoder_input_data = np.zeros_like(X)

In [11]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

In [12]:
# 모델 학습
model.fit([X, decoder_input_data], y, epochs = 50, validation_split = 0.2, callbacks = [early_stopping])

Epoch 1/50
500/500 [==============================] - 7s 9ms/step - loss: 3.3416 - accuracy: 0.0791 - val_loss: 2.6282 - val_accuracy: 0.1665
Epoch 2/50
500/500 [==============================] - 4s 7ms/step - loss: 2.3036 - accuracy: 0.2356 - val_loss: 2.0538 - val_accuracy: 0.2937
Epoch 3/50
500/500 [==============================] - 4s 7ms/step - loss: 1.8835 - accuracy: 0.3524 - val_loss: 1.7223 - val_accuracy: 0.4084
Epoch 4/50
500/500 [==============================] - 4s 7ms/step - loss: 1.6224 - accuracy: 0.4451 - val_loss: 1.5279 - val_accuracy: 0.4764
Epoch 5/50
500/500 [==============================] - 4s 7ms/step - loss: 1.4278 - accuracy: 0.5204 - val_loss: 1.3716 - val_accuracy: 0.5166
Epoch 6/50
500/500 [==============================] - 4s 7ms/step - loss: 1.2847 - accuracy: 0.5747 - val_loss: 1.2411 - val_accuracy: 0.5804
Epoch 7/50
500/500 [==============================] - 4s 9ms/step - loss: 1.1698 - accuracy: 0.6151 - val_loss: 1.1357 - val_accuracy: 0.6227
Epoch 

In [13]:
# 모델 검증 함수
def predict_reverse(input_sequence):
    input_sequence = np.array([char_to_index[char] for char in input_sequence]).reshape((1, 5, 1))
    decoder_input = np.zeros((1, 5, 1))
    predicted_sequence = model.predict([input_sequence, decoder_input])
    predicted_indices = np.argmax(predicted_sequence, axis = -1).reshape((5, ))
    return ''.join([index_to_char[idx] for idx in predicted_indices])

In [14]:
# 검증용 문자열 리스트 (5글자짜리 30개)
test_strings = ["".join(np.random.choice(list(chars), 5)) for _ in range(30)]

In [15]:
# 실제 거꾸로 된 문자열 리스트
expected_outputs = [s[::-1] for s in test_strings]

In [16]:
# 모델 검증 및 정확도 계산
correct_predictions = 0
total_predictions = len(test_strings)

for i, test_string in enumerate(test_strings):
    predicted_output = predict_reverse(test_string)
    is_correct = predicted_output == expected_outputs[i]

    if is_correct:
        correct_predictions += 1

    print(f"입력: {test_string}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {expected_outputs[i]}")
    print(f"정확 여부: {'맞음' if is_correct else '틀림'}\n")

accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 [==============================] - 1s 757ms/step
입력: mPZLj
예측된 출력: jLZPm
실제 출력: jLZPm
정확 여부: 맞음

1/1 [==============================] - 0s 25ms/step
입력: rhyfp
예측된 출력: pfyhr
실제 출력: pfyhr
정확 여부: 맞음

1/1 [==============================] - 0s 32ms/step
입력: Hfydd
예측된 출력: ddyeH
실제 출력: ddyfH
정확 여부: 틀림

1/1 [==============================] - 0s 25ms/step
입력: UlFmY
예측된 출력: YmFlU
실제 출력: YmFlU
정확 여부: 맞음

1/1 [==============================] - 0s 32ms/step
입력: ZzXZr
예측된 출력: rZXzY
실제 출력: rZXzZ
정확 여부: 틀림

1/1 [==============================] - 0s 24ms/step
입력: GmmKn
예측된 출력: nKmmG
실제 출력: nKmmG
정확 여부: 맞음

1/1 [==============================] - 0s 33ms/step
입력: vlHjf
예측된 출력: fjHlv
실제 출력: fjHlv
정확 여부: 맞음

1/1 [==============================] - 0s 24ms/step
입력: EjkkI
예측된 출력: IkkjE
실제 출력: IkkjE
정확 여부: 맞음

1/1 [==============================] - 0s 25ms/step
입력: zVTGo
예측된 출력: oGTVz
실제 출력: oGTVz
정확 여부: 맞음

1/1 [==============================] - 0s 28ms/step
입력: UMyDu
예측된 출력: uDyMU
실제 출력: uDyMU
정확 여부: 맞음
